In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import requests
import json
import requests
import time
from datetime import datetime
from IPython.display import display, clear_output
import numpy as np
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
target_diff = 0.0025
stop_loss = 0.0025
profit = (target_diff / 0.0001)*0.75
loss = (stop_loss / 0.0001)*0.75
version = '10'

In [4]:
filename_date = '100919'

layout = pd.read_csv('../data/' + filename_date + 'mds_out_layout.csv')
layout = layout['features'].values
data = pd.read_csv('../data/' + filename_date + 'mds_out.csv', usecols=layout)
data.shape

(16415, 785)

In [5]:
data['time'].min()

'2017-01-19 21:00:00'

In [6]:
data['time'].max()

'2019-09-24 19:00:00'

In [7]:
num_cols = ['ask_c', 'bid_c']

In [8]:
data['time'] = pd.to_datetime(data['time'])
data = data.sort_values(by='time')
data = data.drop_duplicates(subset=['time'])
data = data.reset_index(drop=True)
data['index'] = data.index
for n in num_cols:
    data[n] = pd.to_numeric(data[n])
data['target_tp'] = data['ask_c'] + target_diff
data['target_sl'] = data['bid_c'] - stop_loss
data = data[~data['target_tp'].isnull()]

In [9]:
data['diff'] = data['target_tp'] - data['target_sl']
data['diff'].describe()

count    16415.000000
mean         0.005155
std          0.000097
min          0.005090
25%          0.005120
50%          0.005130
75%          0.005140
max          0.006500
Name: diff, dtype: float64

In [10]:
# data[['index', 'time', 'volume', 'bid_o', 'bid_h','bid_l','bid_c', 'ask_o', 'ask_h','ask_l','ask_c', 'ask_prev_c', 'bid_prev_c', 'target_tp', 'target_sl', 'g', 'l', 'l_time']].head(40)

In [11]:
data['g']=np.NaN
data['l']=np.NaN
deep=len(data.index)
irange = data['index'].values
tp_good = False
sl_good = False

for i in irange:

    while tp_good == False and sl_good == False:
        show = (i / len(irange))*100
        show = str(round(show, 2))
        show = show + ' - ' + str(datetime.now())
        display(show)
        tp_good = False
        sl_good = False
    #     print(i)

        for r in irange:
    #             print(i, r, tp_good, sl_good)
            if r > i:
                if tp_good == False and sl_good == False:
                    if  data[data['index'] == r]['ask_h'].values[0] >= data[data['index'] == i]['target_tp'].values[0] and not tp_good:
                        data.at[i, 'g'] = data[data['index'] == r]['index'].values[0]
                        data.at[i, 'g_time'] = data[data['index'] == r]['time'].values[0]
                        tp_good = True
                    else:
                        data.at[i, 'g'] = None
                        data.at[i, 'g_time'] =None

                    if  data[data['index'] == r]['bid_l'].values[0] <= data[data['index'] == i]['target_sl'].values[0] and not sl_good:
                        data.at[i, 'l'] = data[data['index'] == r]['index'].values[0]
                        data.at[i, 'l_time'] = data[data['index'] == r]['time'].values[0]
                        sl_good = True
                    else:  
                        data.at[i, 'l'] = None     
                        data.at[i, 'l_time'] = None  
                else:
                    break

        clear_output()    

#     print(datetime.now())

In [12]:
# START WITH FIRST TARGET INDEX BEC TIME ISSUE

if tp_good:
    target_index = data['g'].max()
    good_data = data[~data['g'].isnull()].copy()

if sl_good:
    target_index = data['l'].max()
    good_data = data[~data['g'].isnull()].copy()

tp_good = False
sl_good = False
done = False

counter = 1
while not done:
    show = (target_index / len(irange))*100
    show = str(round(show, 2))
    show = show + ' - ' + str(datetime.now())
    display(show)
    tp_good = False
    sl_good = False

    for r in irange:
        if r > target_index:
            if tp_good == False and sl_good == False:
                if  data[data['index'] == r]['ask_h'].values[0] >= data[data['index'] == target_index]['target_tp'].values[0] and not tp_good:
                    data.at[target_index, 'g'] = data[data['index'] == r]['index'].values[0]
                    data.at[target_index, 'g_time'] = data[data['index'] == r]['time'].values[0]
                    tp_good = True
                else:
                    data.at[target_index, 'g'] = None
                    data.at[target_index, 'g_time'] =None


                if  data[data['index'] == r]['bid_l'].values[0] <= data[data['index'] == target_index]['target_sl'].values[0] and not sl_good:
                    data.at[target_index, 'l'] = data[data['index'] == r]['index'].values[0]
                    data.at[target_index, 'l_time'] = data[data['index'] == r]['time'].values[0]
                    sl_good = True

                else:  
                    data.at[i, 'l'] = None     
                    data.at[i, 'l_time'] = None  
            else:
                break
    
    if tp_good == True:
        target_index = data['g'].max()
        try:
            temp = data[data['g'] == target_index].copy()
            temp2 = data[data['index'] == target_index].copy()

            if counter == 1:
                good_data = temp
                counter += 1
            else:
                good_data = good_data.append(temp)

            if target_index == data['index'].max():
                done = True
        except IndexError:
            done = True

    elif sl_good == True:
        target_index = data['l'].max()
        try:
            temp = data[data['l'] == target_index].copy()
            temp2 = data[data['index'] == target_index].copy()

            if counter == 1:
                good_data = temp
                counter += 1
            else:
                good_data = good_data.append(temp)

            if target_index == data['index'].max():
                done = True
        except IndexError:
            done = True
            
    else:
        done = True


    clear_output()    

In [13]:
# good_mask = (data['g'].isnull()) & (data['l'].isnull())
# good_data = data[~good_mask].copy()
good_data.shape

(2012, 793)

In [14]:
good_data[['time', 'volume', 'bid_o', 'bid_h','bid_l','bid_c', 'ask_o', 'ask_h','ask_l','ask_c', 'target_tp', 'target_sl', 'g', 'l', 'l_time']].head(10)

,time,volume,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,target_tp,target_sl,g,l,l_time
1,2017-01-19 22:00:00,2878,1.06528,1.06871,1.06528,1.06808,1.06545,1.06888,1.06545,1.06821,1.07071,1.06558,NaN,9.0,2017-01-20 06:00:00
9,2017-01-20 06:00:00,5923,1.06627,1.06665,1.06416,1.06459,1.06638,1.06678,1.06428,1.06470,1.06720,1.06209,15.0,NaN,NaT
15,2017-01-20 12:00:00,8486,1.06545,1.06830,1.06543,1.06783,1.06557,1.06842,1.06555,1.06794,1.07044,1.06533,19.0,NaN,NaT
19,2017-01-20 16:00:00,5856,1.06806,1.07087,1.06797,1.07043,1.06820,1.07103,1.06811,1.07062,1.07312,1.06793,25.0,NaN,NaT
25,2017-01-22 22:00:00,4455,1.07233,1.07442,1.07206,1.07436,1.07247,1.07458,1.07222,1.07449,1.07699,1.07186,NaN,37.0,2017-01-23 10:00:00
37,2017-01-23 10:00:00,16754,1.07338,1.07373,1.07087,1.07297,1.07353,1.07386,1.07101,1.07308,1.07558,1.07047,45.0,NaN,NaT
45,2017-01-23 18:00:00,1123,1.07427,1.07682,1.07423,1.07621,1.07443,1.07707,1.07436,1.07657,1.07907,1.07371,NaN,54.0,2017-01-24 03:00:00
54,2017-01-24 03:00:00,6523,1.07544,1.07563,1.07368,1.07405,1.07559,1.07578,1.07385,1.07419,1.07669,1.07155,62.0,NaN,NaT
62,2017-01-24 11:00:00,10717,1.07469,1.07659,1.07428,1.07602,1.07480,1.07673,1.07439,1.07625,1.07875,1.07352,NaN,66.0,2017-01-24 15:00:00
66,2017-01-24 15:00:00,3592,1.07560,1.07577,1.07330,1.07342,1.07575,1.07590,1.07344,1.07355,1.07605,1.07092,84.0,NaN,NaT


In [15]:
def get_target(value):
    if value > 0:
        return 1
    else:
        return 0

In [16]:
# max_date = data['time'].max()
good_data['g_time'] = pd.to_datetime(good_data['g_time'])
good_data['l_time'] = pd.to_datetime(good_data['l_time'])

good_data['g'] = pd.to_numeric(good_data['g'])
good_data['l'] = pd.to_numeric(good_data['l'])

good_data['g'] = good_data['g'].fillna(0)
good_data['l'] = good_data['l'].fillna(0)

# data['g'] = data['g'].fillna(data['index'].max())
# data['l'] = data['l'].fillna(data['index'].max())

good_data['long_target'] = good_data['g'].apply(get_target)
good_data['short_target'] = good_data['l'].apply(get_target)

print(good_data['long_target'].value_counts())
print()
print(good_data['short_target'].value_counts())

1    1012
0    1000
Name: long_target, dtype: int64

1    1016
0     996
Name: short_target, dtype: int64


In [17]:
# good_data['g_time_shift'] = good_data['g_time'].shift(1)
# good_data['g_time_shift'] = pd.to_datetime(good_data['g_time_shift'] )
good_data['time'] = pd.to_datetime(good_data['time'] )
# good_data['g_diff'] = good_data['g_time'] - good_data['g_time_shift']
# good_data['g_diff'] = good_data['g_diff'].dt.seconds

In [18]:
# good_data['order'] = good_data.index
# t1 = good_data[good_data['g'] != 0.0].drop_duplicates(subset='g')
# t2 = good_data[good_data['l'] != 0.0].drop_duplicates(subset='l')
# good_data = pd.concat([t1, t2], axis=0)
# good_data= good_data.sort_values(by='order')
good_data['same_time'] = good_data['g_time'] == good_data['l_time']
good_data['same_time'] = good_data['same_time'].apply(lambda x: 1 if x else 0)
good_data = good_data[good_data['same_time'] == 0]
good_data.shape

(1996, 796)

In [19]:
pos = good_data[good_data['long_target'] == 1].copy()
print(pos.shape)
neg = good_data[good_data['long_target'] == 0].copy()

pos['time_diff'] = (pos['g_time'] - pos['time']) / np.timedelta64(1, 'h')
pos_bad = pos[pos['time_diff'] > 24].copy()
pos_bad['long_target'] = 0

pos = pos[pos['time_diff'] <= 24]

pos.drop(['time_diff'], axis=1, inplace=True)
pos_bad.drop(['time_diff'], axis=1, inplace=True)

good_data = pd.concat([pos, neg, pos_bad], axis=0)
good_data.shape

(996, 796)


(1996, 796)

In [20]:
good_data.to_csv('../data/' + filename_date + '_mds_out_final_v' + version + '_long_live.csv', index=False, header=True)

In [21]:
good_data[['time', 'bid_c', 'ask_c','target_tp', 'target_sl', 'g', 'l', 'g_time', 'l_time', 'long_target', 'short_target', 'time']].head(10)

,time,bid_c,ask_c,target_tp,target_sl,g,l,g_time,l_time,long_target,short_target,time
9,2017-01-20 06:00:00,1.06459,1.06470,1.06720,1.06209,15.0,0.0,2017-01-20 12:00:00,NaT,1,0,2017-01-20 06:00:00
15,2017-01-20 12:00:00,1.06783,1.06794,1.07044,1.06533,19.0,0.0,2017-01-20 16:00:00,NaT,1,0,2017-01-20 12:00:00
37,2017-01-23 10:00:00,1.07297,1.07308,1.07558,1.07047,45.0,0.0,2017-01-23 18:00:00,NaT,1,0,2017-01-23 10:00:00
54,2017-01-24 03:00:00,1.07405,1.07419,1.07669,1.07155,62.0,0.0,2017-01-24 11:00:00,NaT,1,0,2017-01-24 03:00:00
66,2017-01-24 15:00:00,1.07342,1.07355,1.07605,1.07092,84.0,0.0,2017-01-25 09:00:00,NaT,1,0,2017-01-24 15:00:00
86,2017-01-25 11:00:00,1.07262,1.07276,1.07526,1.07012,88.0,0.0,2017-01-25 13:00:00,NaT,1,0,2017-01-25 11:00:00
111,2017-01-26 12:00:00,1.06708,1.06720,1.06970,1.06458,114.0,0.0,2017-01-26 15:00:00,NaT,1,0,2017-01-26 12:00:00
117,2017-01-26 18:00:00,1.06811,1.06839,1.07089,1.06561,133.0,0.0,2017-01-27 10:00:00,NaT,1,0,2017-01-26 18:00:00
133,2017-01-27 10:00:00,1.06978,1.06992,1.07242,1.06728,134.0,0.0,2017-01-27 11:00:00,NaT,1,0,2017-01-27 10:00:00
156,2017-01-30 09:00:00,1.06286,1.06303,1.06553,1.06036,158.0,0.0,2017-01-30 11:00:00,NaT,1,0,2017-01-30 09:00:00


In [22]:
shorts = good_data[good_data['short_target'] == 1].copy()
shorts['time'] = pd.to_datetime(shorts['time'])
shorts['l_time'] = pd.to_datetime(shorts['l_time'] )
shorts['time_diff'] = shorts['l_time'] - shorts['time']
shorts['time_diff'].head() 

1    08:00:00
25   12:00:00
45   09:00:00
62   04:00:00
84   02:00:00
Name: time_diff, dtype: timedelta64[ns]

In [23]:
shorts['time_diff'] = shorts['time_diff']  / np.timedelta64(1, 'h')

In [24]:
shorts['time_diff'].describe()

count    1000.000000
mean       11.197000
std        17.078521
min         1.000000
25%         2.000000
50%         5.000000
75%        13.000000
max       142.000000
Name: time_diff, dtype: float64

In [25]:
dcounts = good_data['time'].value_counts().reset_index()
dcounts.columns = ['time', 'count']